# Stock Analysis Assistant (Adaptive RAG)

Welcome to the **Stock Analysis Assistant**, an intelligent application that combines Retrieval-Augmented Generation (RAG) with real-time web and financial data.

## What it does:
- Answers stock-related queries using both **internal company data** and **real-time news & sentiment**
- Uses **adaptive RAG** to decide whether to search web sources
- Supports natural language questions like:
  - *"What does Nvidia do and when are its earnings?"*
  - *"Any recent news on Tesla?"*
  - *"Summarize Apple's financial performance and market sentiment."*

---

## Tech Stack:
- **LangChain + Mistral/Groq** – For LLM-based Q&A
- **ChromaDB** – Internal vector store containing all listed companies' overview
- **HuggingFace Embeddings** – For semantic retrieval
- **Hugging Face Hub** – For storing the vector database (cloud-based persistence)
- **Tavily API** – For real-time web search (news, sentiment, etc.)
- **Alpha Vantage API** *(optional)* – For structured financial data
- **LangSmith** – Observability, tracing, and debugging
- **Streamlit (Planned)** – To deploy an interactive web app

---
##  Key Features:
- **Query rewriting & multi-query expansion** to improve retrieval
- **Fusion of internal + web results** with semantic reranking
- **Adaptive logic** to trigger web search only when needed
- **LangSmith Tracing** for full pipeline observability

---

## Storage & Deployment:
- Vector DB stored on **Hugging Face Hub**
- Planned deployment using **Streamlit** for live demos

---

## Potential Areas for Improvement:
- **Better routing** for information retrieval for different question types.
- **Structured Answers** based on the questions' classification i.e. Fundamental Analysis/Technical Analysis/News Sentiment.
- Add **cross-encoder re-ranking** to improve document relevance
- Create an **evaluation dataset** for quantitative benchmarking
- Integrate **financial charts or visualizations**
- Add **agentic capabilities** for multi-step queries
- Implement **user feedback loop** for adaptive learning


---

> This project showcases an end-to-end RAG pipeline with real-time web and financial data — optimized for clarity, performance, and interactivity.

In [1]:
%%capture
!pip install langchain_groq langsmith alpha_vantage
!pip install -U langchain-huggingface langchain-chroma
!pip install -U langchain langchain-openai

In [2]:
#LangSmith Tracing
import os
from dotenv import load_dotenv

load_dotenv()  # Loads from .env file


from langsmith import  Client
from langchain_core.runnables import RunnableLambda, RunnableSequence

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "stock-analysis rag project"

In [3]:
# --- Tavily and alpha_vantage API Key ---
from langchain_community.tools.tavily_search import TavilySearchResults
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.fundamentaldata import FundamentalData

os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")
os.environ["ALPHA_VANTAGE_API_KEY"] = os.getenv("ALPHA_VANTAGE_API_KEY")

In [2]:

# --- Embeddings + Vector Store ---

from langchain_huggingface import HuggingFaceEmbeddings  
from langchain_chroma import Chroma 
from huggingface_hub import snapshot_download

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

def load_vectorstore():
    # Download company_vectors dataset from HuggingFace
    dataset_path = snapshot_download(
        repo_id="gargumang411/company_vectors",
        repo_type="dataset",
        cache_dir="./company_vectors_cache"
    )
    embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    return Chroma(
        persist_directory=dataset_path,
        embedding_function=embedding_model
    )

# Initialize vectorstore
vectorstore = load_vectorstore()


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/2.46k [00:00<?, ?B/s]

company_vectors.zip:   0%|          | 0.00/72.0M [00:00<?, ?B/s]

In [5]:

# --- LLM ---
from langchain_groq import ChatGroq
# from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

llm = ChatGroq(
    model="llama3-70b-8192",  # or "mistral-7b"
    temperature=0,
    groq_api_key= os.getenv("GROQ_API_KEY")
)

# --- LangSmith Client ---
ls_client = Client()

In [6]:
# --- Prompt Templates ---
qa_prompt = PromptTemplate.from_template("""
You are a financial assistant. Below is the context, including intermediate steps (query variants, retrieved documents, summaries) from web sources, a company database, and Alpha Vantage API. Answer the user's question clearly and concisely, citing sources (Company DB, Web, Alpha Vantage) where relevant. For news queries, prioritize recent events (e.g., 2025). For analysis queries, include key financial metrics (e.g., P/E, EPS, revenue) if available. If data is limited, provide a general response.

Intermediate Steps:
- Query Variants: {query_variants}
- Retrieved Documents: {doc_summaries}

Context:
{context}

Question: {question}
Answer:
""")

summarize_prompt = PromptTemplate.from_template("""
Summarize the following financial document to ~200 words, focusing on recent news, financial metrics (e.g., P/E, EPS, revenue), or analyst recommendations. Exclude irrelevant details (e.g., ads, unrelated companies):

{document}
Summary:
""")


In [7]:
from typing import List, Dict, Tuple
from langchain.schema import Document
from collections import defaultdict
import numpy as np
import re
import time
# --- Query Cleaning ---
def clean_query(query: str) -> str:
    query = re.sub(r'[^\w\s\-\.]', '', query)  # Remove special characters
    query = " ".join(query.split()[:20])  # Limit to 20 words
    return query.strip()

# --- Ticker Map ---
def build_ticker_map(vectorstore) -> Dict[str, str]:
    ticker_map = {}
    docs = vectorstore.get()
    for metadata in docs.get("metadatas", []):
        if metadata.get("ticker") and metadata.get("company_name"):
            ticker_map[metadata["company_name"].lower()] = metadata["ticker"]
    return ticker_map

# --- Ticker Extraction ---
def extract_ticker(query: str, vectorstore, ticker_map: Dict[str, str]) -> str:
    query_lower = query.lower()
    for company, ticker in ticker_map.items():
        if company in query_lower:
            return ticker
    search_results = vectorstore.similarity_search(query, k=1)
    if search_results and "ticker" in search_results[0].metadata:
        return search_results[0].metadata["ticker"]
    prompt = f"Extract the stock ticker from this query, return only the ticker symbol or 'TSLA' if unclear:\n\"{query}\""
    return llm.invoke(prompt).content.strip()


extract_ticker_lambda = RunnableLambda(

    lambda inputs: extract_ticker(inputs["query"], inputs["vectorstore"], inputs["ticker_map"])

)
# --- Query Rewriting ---
def translate_query(query: str, ticker: str ) -> str:
    prompt = f"""
You're optimizing user queries for better retrieval in a stock research assistant.

Example:
Original: "any latest news on {ticker}?"
Improved: "{ticker} latest news 2025 earnings stock performance"

Rewrite the user's query to maximize document retrieval relevance for {ticker}, focusing on financial news or metrics:
\"{query}\"
"""
    return clean_query(llm.invoke(prompt).content.strip())


translate_query_lambda = RunnableLambda(

    lambda inputs: translate_query(inputs["query"], inputs["ticker"])

)
# --- Multi-query Generation ---
def generate_query_variants(query: str, n=2) -> List[str]:
    prompt = f"Generate {n} alternative phrasings for this financial query, focusing on recent news or financial metrics:\n\n\"{query}\""
    raw = llm.invoke(prompt).content
    variants = [clean_query(line.strip("-• ").strip()) for line in raw.split("\n") if line.strip()]
    return variants[:n]

generate_query_variants_lambda = RunnableLambda(

    lambda inputs: generate_query_variants(inputs["translated_query"], n=2)

)
# --- Query Classification for Web Search - currently always true---
# def needs_web_search(query: str) -> bool:
#     query_lower = query.lower()
    
#     # If query is long, assume it's complex and needs web search
#     if len(query.split()) > 8:
#         return True

#     # If it's a basic company overview question, no need for web search
#     if query_lower.startswith("what does") and "do" in query_lower:
#         return False
#     if "company overview" in query_lower:
#         return False
    
#     return True
# @traceable
# --- Alpha Vantage Data ---
def fetch_alpha_vantage_data(ticker: str) -> Dict:
    for attempt in range(2):
        try:
            ts = TimeSeries(key=os.environ["ALPHA_VANTAGE_API_KEY"], output_format='json')
            fd = FundamentalData(key=os.environ["ALPHA_VANTAGE_API_KEY"], output_format='json')
            quote_data, _ = ts.get_quote_endpoint(symbol=ticker)
            overview, _ = fd.get_company_overview(symbol=ticker)
            return {
                "price": quote_data.get("05. price"),
                "pe_ratio": overview.get("PERatio"),
                "eps": overview.get("EPS"),
                "revenue": overview.get("RevenueTTM"),
                "market_cap": overview.get("MarketCapitalization")
            }
        except Exception as e:
            print(f"❌ Alpha Vantage API call failed for {ticker} (attempt {attempt+1}): {e}")
            if "rate limit" in str(e).lower():
                time.sleep(5)
            else:
                time.sleep(1)
    return {}

fetch_alpha_vantage_lambda = RunnableLambda(

    lambda inputs: fetch_alpha_vantage_data(inputs["ticker"])

)



# --- Web Search Retrieval ---
def retrieve_docs_with_fusion(query: str, ticker: str, av_data: Dict, k=3) -> Tuple[List[Document], List[str], str]:
    # --- Vector DB Retrieval ---
    internal_docs = []
    results = vectorstore.similarity_search_with_score(query, k=2)
    for doc, score in results:
        internal_docs.append((doc, score + 0.2))

    # --- Web Search Retrieval ---
    web_docs = []
    translated = translate_query(query, ticker)
    variants = generate_query_variants(translated, n=2)
    all_web_queries = [translated] + variants
    query_emb = np.array(embedding_model.embed_query(query))

 

    for q in all_web_queries:
        for attempt in range(2):
            try:
                search = TavilySearchResults(max_results=2, api_key=os.environ["TAVILY_API_KEY"])
                web_results = search.invoke({"query": q})
                if not isinstance(web_results, list):
                    print(f"⚠️ Tavily returned non-list response for query '{q}':", web_results)
                    continue
                for result in web_results:
                    if isinstance(result, dict) and "content" in result and "url" in result:
                        content = " ".join(result["content"].split()[:800])
                        summarized = summarize_document(content)
                        doc = Document(page_content=summarized, metadata={"source": result["url"]})
                        doc_emb = np.array(embedding_model.embed_documents([summarized])[0])
                        distance = np.linalg.norm(query_emb - doc_emb)
                        web_docs.append((doc, distance))
                break
            except Exception as e:
                print(f"❌ Tavily API call failed for query '{q}' (attempt {attempt+1}): {e}")
                if "rate limit" in str(e).lower():
                    time.sleep(2)
                else:
                    time.sleep(1)

    # --- Alpha Vantage Data ---
    all_docs = internal_docs + web_docs
    if av_data:
        av_content = f"Alpha Vantage Data for {ticker}: Price: {av_data['price']}, P/E: {av_data['pe_ratio']}, EPS: {av_data['eps']}, Revenue: {av_data['revenue']}, Market Cap: {av_data['market_cap']}"
        av_doc = Document(page_content=av_content, metadata={"source": "Alpha Vantage"})
        all_docs.append((av_doc, 0.0))  # Prioritize AV data

    # --- Re-Rank with Keyword Scoring ---
    financial_keywords = ["earnings", "revenue", "eps", "p/e", "valuation", "analyst", "rating", "buy", "sell", "news"]
    ranked_docs = []
    for doc, score in all_docs:
        keyword_score = sum(1 for kw in financial_keywords if kw in doc.page_content.lower())
        adjusted_score = score - (keyword_score * 0.1)
        ranked_docs.append((doc, adjusted_score))


    ranked_docs_sorted = sorted(ranked_docs, key=lambda x: x[1])
    top_docs = [doc for doc, _ in ranked_docs_sorted[:k]]
    doc_summaries = "\n".join([f"Source: {doc.metadata.get('source', 'Company DB')}\n{doc.page_content[:100]}..." for doc in top_docs])
    return top_docs, all_web_queries, doc_summaries

retrieve_docs_lambda = RunnableLambda(

    lambda inputs: retrieve_docs_with_fusion(inputs["query"], inputs["ticker"], inputs["av_data"], k=3)

)
# --- Summarize Document ---
def summarize_document(content: str) -> str:
    prompt = summarize_prompt.format(document=content)
    summary = llm.invoke(prompt).content.strip()
    return " ".join(summary.split()[:200])

summarize_document_lambda = RunnableLambda(summarize_document)

In [8]:
# --- Final RAG Chain  ---
class FusionRAG:
    def __init__(self, llm, vectorstore, docs_per_query=3):
        self.llm = llm
        self.vectorstore = vectorstore
        self.docs_per_query = docs_per_query
        self.ticker_map = build_ticker_map(vectorstore)
        # Define the RAG pipeline as a RunnableSequence

        self.pipeline = RunnableSequence(

            # Step 1: Extract ticker

            lambda inputs: {

                **inputs,

                "ticker": extract_ticker_lambda.invoke({

                    "query": inputs["query"],

                    "vectorstore": self.vectorstore,

                    "ticker_map": self.ticker_map

                })

            },

            # Step 2: Translate query

            lambda inputs: {

                **inputs,

                "translated_query": translate_query_lambda.invoke({

                    "query": inputs["query"],

                    "ticker": inputs["ticker"]

                })

            },

            # Step 3: Generate query variants

            lambda inputs: {

                **inputs,

                "query_variants": generate_query_variants_lambda.invoke({

                    "translated_query": inputs["translated_query"]

                })

            },

            # Step 4: Fetch Alpha Vantage data

            lambda inputs: {

                **inputs,

                "av_data": fetch_alpha_vantage_lambda.invoke({

                    "ticker": inputs["ticker"]

                })

            },

            # Step 5: Retrieve documents

            lambda inputs: {

                **inputs,

                "docs": retrieve_docs_lambda.invoke({

                    "query": inputs["query"],

                    "ticker": inputs["ticker"],

                    "av_data": inputs["av_data"]

                })[0],

                "query_variants": retrieve_docs_lambda.invoke({

                    "query": inputs["query"],

                    "ticker": inputs["ticker"],

                    "av_data": inputs["av_data"]

                })[1],

                "doc_summaries": retrieve_docs_lambda.invoke({

                    "query": inputs["query"],

                    "ticker": inputs["ticker"],

                    "av_data": inputs["av_data"]

                })[2]

            },

            # Step 6: Generate final answer

            lambda inputs: {

                "result": self.llm.invoke(qa_prompt.format(

                    context="\n\n".join([f"Source: {'Web' if 'source' in doc.metadata else 'Company DB'}\n{doc.page_content}" for doc in inputs["docs"]]),

                    query_variants=str(inputs["query_variants"]),

                    doc_summaries=inputs["doc_summaries"],

                    question=inputs["query"]

                )).content.strip(),

                "query_variants": inputs["query_variants"],

                "doc_summaries": inputs["doc_summaries"]

            }

        )



    
    

    def invoke(self, inputs: Dict[str, str]) -> Dict[str, str]:
        query = inputs["query"]
        result = self.pipeline.invoke({"query": query})

        # Log final response to LangSmith dataset
        dataset_name = "StockRAG"
        # Check if dataset exists and get its UUID
        existing_datasets = ls_client.list_datasets()
        dataset_id = None
        for dataset in existing_datasets:
            if dataset.name == dataset_name:
                dataset_id = dataset.id
                break
        if dataset_id is None:
            # Create dataset and get its UUID
            dataset = ls_client.create_dataset(dataset_name=dataset_name)
            dataset_id = dataset.id
        ls_client.create_examples(
            dataset_id=dataset_id,
            inputs=[{"query": query}],
            outputs=[{"answer": result["result"]}]
        )

        return result

In [9]:
# --- Run ---
qa_chain = FusionRAG(llm=llm, vectorstore=vectorstore)
query = "Fundamental Analysis of SMCI"
response = qa_chain.invoke({"query": query})
print("\n💬 Answer:\n", response["result"])


💬 Answer:
 Based on the provided data and summaries, here is a fundamental analysis of SMCI:

**Valuation:** SMCI's current market price is $31.505, with a P/E ratio of 13.7 and EPS of $2.3 (Alpha Vantage). According to one analysis, the intrinsic value of SMCI is $44.635, suggesting that the stock is undervalued (Web).

**Revenue Growth:** Analysts expect 70% revenue growth in fiscal year 2025, reaching $25 billion in revenue, and 20-25% growth in fiscal year 2026 (Web).

**Earnings Growth:** EPS is expected to grow by 48% in fiscal year 2025, reaching $2.97, and continue to grow in fiscal year 2026 (Web).

**Financial Health:** SMCI's financial health appears strong, with a debt-to-equity ratio of 0.40 and a gross profit margin of 14% (Web).

**Analyst Recommendations:** The analysis implies a "Buy" recommendation, as the intrinsic value is higher than the current market price, suggesting that the stock has upside potential (Web).

Overall, SMCI's financial statements indicate stron

In [10]:
query = "What does Amazon do? give detailed answer"
response = qa_chain.invoke({"query": query})
print("\n💬 Answer:\n", response["result"])


💬 Answer:
 According to the Company DB, Amazon.com, Inc. (AMZN) is a multifaceted company that engages in various business activities. Here's a detailed breakdown of what Amazon does:

1. **Retail Sale of Consumer Products**: Amazon sells a wide range of consumer products through its online and physical stores in North America and internationally.
2. **Advertising Services**: The company offers various advertising services, including sponsored ads, display, and video advertising, to help businesses reach their target audience.
3. **Subscriptions Services**: Amazon provides subscription-based services, such as Amazon Prime, which offers benefits like free shipping, streaming of music and video content, and other perks.
4. **Manufacturing and Sales of Electronic Devices**: Amazon designs, manufactures, and sells electronic devices, including Kindle e-readers, Fire tablets, Fire TVs, Echo smart speakers, Ring doorbells, Blink security cameras, and Eero Wi-Fi routers.
5. **Media Content D

In [11]:
# def retrieve_docs_internal_db(query, k=3):
#     return vectorstore.similarity_search(query, k=k)


# def rag_answer(query):
#     docs = retrieve_docs_internal_db(query)
#     prompt = format_prompt(docs, query)
#     response = llm.invoke(prompt)
#     return response.content

In [12]:
# import os
# from typing import List, Dict, Tuple
# from collections import defaultdict
# import numpy as np
# import requests
# from langchain_community.vectorstores import Chroma
# from langchain_community.embeddings import HuggingFaceEmbeddings
# from langchain_groq import ChatGroq
# from langchain.prompts import PromptTemplate
# from langchain.schema import Document
# from langchain_community.tools.tavily_search import TavilySearchResults
# from sentence_transformers import CrossEncoder

# # --- Environment Variables ---
# # os.environ["LANGCHAIN_TRACING_V2"] = "true"
# # os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# # os.environ["LANGCHAIN_API_KEY"] = "your-langsmith-key"
# # os.environ["LANGCHAIN_PROJECT"] = "your-project-name"
# # os.environ["TAVILY_API_KEY"] = "your-tavily-api-key"
# os.environ["ALPHA_VANTAGE_API_KEY"] = "FP9PA5A2IF03ZK6Y"

# # --- Embeddings + Vector DB ---
# embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
# vectorstore = Chroma(persist_directory="company_vectors", embedding_function=embedding_model)

# # --- Cross-Encoder for Re-Ranking ---
# cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

# # --- LLM Setup ---
# # llm = ChatGroq(
# #     model="llama3-70b-8192",
# #     temperature=0,
# #     groq_api_key="your-groq-api-key"
# # )

# # --- Prompt Templates ---
# qa_prompt = PromptTemplate.from_template("""
# You are a financial assistant. Based on the following information, which includes company data and web sources, provide a detailed fundamental analysis for the user's question, including key metrics (e.g., P/E, EPS, revenue growth, debt-to-equity) and a buy/sell recommendation if requested. Cite sources (Company DB, Web, or Alpha Vantage) where relevant. If data is limited, provide a general analysis based on available information.

# Context:
# {context}

# Question: {question}
# Answer:
# """)

# summarize_prompt = PromptTemplate.from_template("""
# Summarize the following financial document to ~200 words, focusing on key fundamental metrics (e.g., P/E, EPS, revenue, debt-to-equity) and analyst recommendations:

# {document}
# Summary:
# """)

# classify_prompt = PromptTemplate.from_template("""
# Classify the intent of this financial query as one of: 'fundamental_analysis', 'company_overview', 'news', 'technical_analysis', or 'other'. Return only the intent label.

# Query: {query}
# """)

# # --- Query Rewriting ---
# def translate_query(query: str) -> str:
#     prompt = f"Rewrite this stock-related query to improve document retrieval, focusing on financial metrics and analysis:\n\n\"{query}\""
#     return llm.invoke(prompt).content.strip()

# # --- Multi-query Generation ---
# def generate_query_variants(query: str, n=2) -> List[str]:
#     prompt = f"Generate {n} alternative phrasings for this financial query, emphasizing fundamental analysis:\n\n\"{query}\""
#     raw = llm.invoke(prompt).content
#     return [line.strip("-• ").strip() for line in raw.split("\n") if line.strip()][:n]

# # --- Query Classification ---
# def classify_query_intent(query: str) -> str:
#     prompt = classify_prompt.format(query=query)
#     return llm.invoke(prompt).content.strip()

# # --- Fetch Alpha Vantage Data ---
# def fetch_alpha_vantage_data(ticker: str) -> str:
#     url = f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={ticker}&apikey={os.environ['ALPHA_VANTAGE_API_KEY']}"
#     try:
#         response = requests.get(url)
#         data = response.json()
#         if "Symbol" not in data:
#             return ""
#         metrics = (
#             f"Ticker: {data['Symbol']}\n"
#             f"P/E Ratio: {data.get('PERatio', 'N/A')}\n"
#             f"EPS: {data.get('EPS', 'N/A')}\n"
#             f"Revenue (TTM): {data.get('RevenueTTM', 'N/A')}\n"
#             f"Debt-to-Equity: {data.get('DebtToEquity', 'N/A')}\n"
#             f"Market Cap: {data.get('MarketCapitalization', 'N/A')}\n"
#             f"Analyst Target Price: {data.get('AnalystTargetPrice', 'N/A')}"
#         )
#         return metrics
#     except:
#         return ""

# # --- Summarize Document ---
# def summarize_document(content: str) -> str:
#     prompt = summarize_prompt.format(document=content)
#     summary = llm.invoke(prompt).content.strip()
#     return " ".join(summary.split()[:200])  # Ensure ~200 words

# # --- Adaptive Retrieval with Fusion, Web Search, and Re-Ranking ---
# def retrieve_docs_with_fusion(query: str, ticker: str = "TSLA", k=5) -> List[Document]:
#     intent = classify_query_intent(query)
#     docs = []

#     # Internal retrieval with fusion
#     translated = translate_query(query)
#     variants = generate_query_variants(translated, n=2)
#     all_queries = [query, translated] + variants
#     doc_scores_internal = defaultdict(list)
#     doc_objects_internal = {}
#     for q in all_queries:
#         results: List[Tuple[Document, float]] = vectorstore.similarity_search_with_score(q, k=k)
#         for doc, score in results:
#             key = doc.metadata.get("ticker", doc.page_content[:50])
#             doc_scores_internal[key].append(score)
#             doc_objects_internal[key] = doc
#     internal_docs = [(doc_objects_internal[key], sum(scores) / len(scores)) for key, scores in doc_scores_internal.items()]

#     # Web search and Alpha Vantage for fundamental analysis or news
#     if intent in ["fundamental_analysis", "news", "technical_analysis"]:
#         # Tavily search
#         search = TavilySearchResults(max_results=k)
#         web_query = f"{ticker} {intent.replace('_', ' ')} 2025"
#         web_results = search.invoke({"query": web_query})
#         query_emb = np.array(embedding_model.embed_query(query))
#         web_docs = []
#         for result in web_results:
#             content = " ".join(result["content"].split()[:800])
#             summarized = summarize_document(content)
#             doc = Document(page_content=summarized, metadata={"source": result["url"]})
#             doc_emb = np.array(embedding_model.embed_documents([summarized])[0])
#             distance = np.linalg.norm(query_emb - doc_emb)
#             web_docs.append((doc, distance))

#         # Alpha Vantage data
#         av_data = fetch_alpha_vantage_data(ticker)
#         if av_data:
#             summarized = summarize_document(av_data)
#             doc = Document(page_content=summarized, metadata={"source": "Alpha Vantage"})
#             doc_emb = np.array(embedding_model.embed_documents([summarized])[0])
#             distance = np.linalg.norm(query_emb - doc_emb)
#             web_docs.append((doc, distance))

#         docs.extend(web_docs)

#     # Combine internal and web docs
#     docs.extend(internal_docs)
#     if not docs:
#         return []

#     # Re-rank with cross-encoder
#     query_doc_pairs = [(query, doc[0].page_content) for doc in docs]
#     scores = cross_encoder.predict(query_doc_pairs)
#     ranked_docs = sorted(zip([doc[0] for doc in docs], scores), key=lambda x: x[1], reverse=True)
#     top_docs = [doc for doc, _ in ranked_docs[:k]]
#     return top_docs

# # --- Final RAG Answer Generator ---
# class FusionRAG:
#     def __init__(self, llm, docs_per_query=5):
#         self.llm = llm
#         self.docs_per_query = docs_per_query

#     def invoke(self, inputs: Dict[str, str]) -> Dict[str, str]:
#         query = inputs["query"]
#         ticker = "TSLA"  # Hardcoded for Tesla; modify for dynamic ticker extraction
#         docs = retrieve_docs_with_fusion(query, ticker, k=self.docs_per_query)
#         if not docs:
#             fallback_prompt = qa_prompt.format(context="No specific financial data retrieved.", question=query)
#             result = self.llm.invoke(fallback_prompt).content.strip()
#             return {"result": result}
#         context = "\n\n".join([f"Source: {'Web' if 'source' in doc.metadata else 'Company DB'}\n{doc.page_content}" for doc in docs])
#         prompt = qa_prompt.format(context=context, question=query)
#         result = self.llm.invoke(prompt).content.strip()
#         return {"result": result}

# # --- Run Example ---
# qa_chain = FusionRAG(llm=llm)
# query = "Can you give Tesla's fundamental analysis. Is it a buy"
# response = qa_chain.invoke({"query": query})
# print("\n💬 Answer:\n", response["result"])

In [13]:
# # --- Run Example ---
# qa_chain = FusionRAG(llm=llm)
# query = "can you fetch the 10-year P/E ratio for Tesla."
# response = qa_chain.invoke({"query": query})
# print("\n💬 Answer:\n", response["result"])